In [69]:
import warnings; warnings.simplefilter('ignore')
import pandas                  as pd
import numpy                   as np
import sklearn.model_selection as ms
import sklearn.metrics         as mt

from imblearn.under_sampling   import RandomUnderSampler
from sklearn.ensemble          import RandomForestClassifier

print('All fine!')

All fine!


In [70]:
instances_pe = pd.read_csv('./pe_dataset.csv')
print('PE dataset:', instances_pe.shape)

instances_sp = pd.read_csv('./sp-dataset.csv')
print('SP dataset:', instances_sp.shape)

PE dataset: (81399, 12)
SP dataset: (67683, 12)


In [71]:
cln_instances_pe = instances_pe.drop(columns=['panel_info', 'panel_eplet'])
print('PE cleaned dataset:', cln_instances_pe.shape)

cln_instances_sp = instances_sp.drop(columns=['panel_info', 'panel_eplet'])
print('SP cleaned dataset:', cln_instances_sp.shape)


PE cleaned dataset: (81399, 10)
SP cleaned dataset: (67683, 10)


In [72]:

cln_instances_pe.head(3)

,locus_abc,locus_dr,locus_dq,locus_dp,panel_nc,panel_pc,panel_allele_count,panel_min_mfi,panel_max_mfi,reactive
0,1,0,0,0,126,10803,1,483,483,0
1,1,0,0,0,126,10803,1,778,778,0
2,1,0,0,0,126,10803,4,6939,10219,1


In [73]:
eplets_pe_count = cln_instances_pe.reactive.value_counts()

print('Non reactive:', eplets_pe_count[0])
print('Reactive:    ', eplets_pe_count[1])
print('Proportion:  ', int(eplets_pe_count[0] / eplets_pe_count[1]), ': 1')

Non reactive: 77981
Reactive:     3418
Proportion:   22 : 1


In [74]:
cln_instances_sp.head(3)

,locus_abc,locus_dr,locus_dq,locus_dp,panel_nc,panel_pc,panel_allele_count,panel_min_mfi,panel_max_mfi,reactive
0,1,0,0,0,138,12707,1,0,0,0
1,1,0,0,0,138,12707,27,0,17344,0
2,1,0,0,0,138,12707,1,86,86,0


In [75]:

eplets_sp_count = cln_instances_sp.reactive.value_counts()

print('Non reactive:', eplets_sp_count[0])
print('Reactive:    ', eplets_sp_count[1])
print('Proportion:  ', int(eplets_sp_count[0] / eplets_sp_count[1]), ': 1')

Non reactive: 65870
Reactive:     1813
Proportion:   36 : 1


In [76]:
from imblearn.under_sampling import RandomUnderSampler

In [77]:
imb_train_labels_pe = np.array(cln_instances_pe['reactive'])
imb_instances_pe = cln_instances_pe.drop(columns=['reactive'])
rus = RandomUnderSampler()
# Use fit_resample instead of fit_sample
train_instances_pe, train_labels_pe = rus.fit_resample(imb_instances_pe, imb_train_labels_pe)

In [78]:
print('Train labels (PE)', train_instances_pe.shape)
print('Train instances (PE):', train_labels_pe.shape)

Train labels (PE) (6836, 9)
Train instances (PE): (6836,)


In [79]:
imb_train_labels_sp = np.array(cln_instances_sp['reactive'])
imb_instances_sp = cln_instances_sp.drop(columns=['reactive'])
rus=RandomUnderSampler()
train_instances_sp, train_labels_sp = rus.fit_resample(imb_instances_sp, imb_train_labels_sp)

print('Train labels (SP)', train_instances_sp.shape)
print('Train instances (SP):', train_labels_sp.shape)

Train labels (SP) (3626, 9)
Train instances (SP): (3626,)


In [80]:
validation_labels_sp = np.array(cln_instances_sp['reactive'])
validation_instances_sp = cln_instances_sp.drop(columns=['reactive'])

print('Validation labels (SP)', validation_instances_sp.shape)
print('Validation instances (SP):', validation_labels_sp.shape)

Validation labels (SP) (67683, 9)
Validation instances (SP): (67683,)


In [81]:
#  Simple Test and Validation
clf = RandomForestClassifier(n_estimators=100)

clf.fit(train_instances_pe, train_labels_pe)

feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = cln_instances_pe.drop(columns=['reactive']).columns,
                                   columns = ['importance']).sort_values('importance', ascending=False)

print('Variable importance:\n\n', feature_importances)

predicted_labels_sp = clf.predict(validation_instances_sp)

print('- Confusion matrix (TN FP FN TP):', mt.confusion_matrix(validation_labels_sp, predicted_labels_sp).ravel())

print()
print('Validation metrics:\n')
print(' - AUC-ROC:  %0.2f' % (mt.roc_auc_score(validation_labels_sp, predicted_labels_sp) * 100))
print(' - Accuracy: %0.2f' % (mt.accuracy_score(validation_labels_sp, predicted_labels_sp) * 100))


Variable importance:

                     importance
panel_min_mfi         0.515562
panel_max_mfi         0.203921
panel_allele_count    0.089555
panel_nc              0.081381
panel_pc              0.080143
locus_abc             0.008243
locus_dq              0.007618
locus_dp              0.007463
locus_dr              0.006114
- Confusion matrix (TN FP FN TP): [57617  8253   163  1650]

Validation metrics:

 - AUC-ROC:  89.24
 - Accuracy: 87.57


In [82]:
#Cross Validation
clf = RandomForestClassifier(n_estimators=100)
cv = 5
scoring = ['roc_auc', 'accuracy']

print('\nPE ---')

predicted_labels_pe = ms.cross_val_predict(clf, train_instances_pe, train_labels_pe, cv=cv, n_jobs=-1)

print()
print('First step:\n')
print('- Confusion matrix (TN FP FN TP):', mt.confusion_matrix(train_labels_pe, predicted_labels_pe).ravel())
print("- AUC-ROC: %0.2f" % (mt.roc_auc_score(train_labels_pe, predicted_labels_pe)*100))
print("- Accuracy: %0.2f" % (mt.accuracy_score(train_labels_pe, predicted_labels_pe)*100))

scores = ms.cross_validate(clf, train_instances_pe, train_labels_pe, cv=cv, n_jobs=-1, scoring=scoring)

print()
print('Second step:\n')
print("- AUC-ROC:  %0.2f (+/- %0.2f)" % (scores['test_roc_auc'].mean()*100, scores['test_roc_auc'].std()*100))
print("- Accuracy: %0.2f (+/- %0.2f)" % (scores['test_accuracy'].mean()*100, scores['test_accuracy'].std()*100))



PE ---

First step:

- Confusion matrix (TN FP FN TP): [2717  701  447 2971]
- AUC-ROC: 83.21
- Accuracy: 83.21

Second step:

- AUC-ROC:  88.14 (+/- 1.24)
- Accuracy: 83.62 (+/- 1.38)


In [83]:
# Cross-validation & feature selection
clf = RandomForestClassifier(n_estimators=100)
cv = 5
scoring = ['roc_auc', 'accuracy']

print('\nPE ---')

predicted_labels_pe = ms.cross_val_predict(clf, train_instances_pe, train_labels_pe, cv=cv, n_jobs=-1)

print()
print('First step:\n')
print('- Confusion matrix (TN FP FN TP):', mt.confusion_matrix(train_labels_pe, predicted_labels_pe).ravel())
print("- AUC-ROC: %0.2f" % (mt.roc_auc_score(train_labels_pe, predicted_labels_pe)*100))
print("- Accuracy: %0.2f" % (mt.accuracy_score(train_labels_pe, predicted_labels_pe)*100))

scores = ms.cross_validate(clf, train_instances_pe, train_labels_pe, cv=cv, n_jobs=-1, scoring=scoring)

print()
print('Second step:\n')
print("- AUC-ROC:  %0.2f (+/- %0.2f)" % (scores['test_roc_auc'].mean()*100, scores['test_roc_auc'].std()*100))
print("- Accuracy: %0.2f (+/- %0.2f)" % (scores['test_accuracy'].mean()*100, scores['test_accuracy'].std()*100))


#---


print('\nSP ---')

predicted_labels_sp = ms.cross_val_predict(clf, train_instances_sp, train_labels_sp, cv=cv, n_jobs=-1)

print()
print('First step:\n')
print('- Confusion matrix (TN FP FN TP):', mt.confusion_matrix(train_labels_sp, predicted_labels_sp).ravel())
print("- AUC-ROC: %0.2f" % (mt.roc_auc_score(train_labels_sp, predicted_labels_sp)*100))
print("- Accuracy: %0.2f" % (mt.accuracy_score(train_labels_sp, predicted_labels_sp)*100))

scores = ms.cross_validate(clf, train_instances_sp, train_labels_sp, cv=cv, n_jobs=-1, scoring=scoring)

print()
print('Second step:\n')
print("- AUC-ROC:  %0.2f (+/- %0.2f)" % (scores['test_roc_auc'].mean()*100, scores['test_roc_auc'].std()*100))
print("- Accuracy: %0.2f (+/- %0.2f)" % (scores['test_accuracy'].mean()*100, scores['test_accuracy'].std()*100))




PE ---

First step:

- Confusion matrix (TN FP FN TP): [2719  699  438 2980]
- AUC-ROC: 83.37
- Accuracy: 83.37

Second step:

- AUC-ROC:  88.10 (+/- 1.38)
- Accuracy: 82.99 (+/- 1.46)

SP ---

First step:

- Confusion matrix (TN FP FN TP): [1545  268  139 1674]
- AUC-ROC: 88.78
- Accuracy: 88.78

Second step:

- AUC-ROC:  92.64 (+/- 1.11)
- Accuracy: 88.91 (+/- 0.63)


In [84]:
# Cross-validation & feature selection & hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

clf = RandomForestClassifier()
cv = 5
scoring = ['roc_auc', 'accuracy']

print()
print('Parameter names currently in use by RF:\n')
print(clf.get_params())

param_grid = {
  'n_estimators': [100, 200, 400],
  'criterion': ['gini', 'entropy'],
  'min_samples_split': [2, 4, 8],
  'min_samples_leaf': [1, 2, 4],
  'max_features': [None, 'auto', 'sqrt', 'log2']
}

print('\nPE ---')

params_search = RandomizedSearchCV(estimator=clf, param_distributions=param_grid, n_iter=cv, scoring='roc_auc', n_jobs=3, cv=(cv*cv))
params_search.fit(train_instances_pe, train_labels_pe)

print()
print('Best parameters:\n')
print(params_search.best_params_)

predicted_labels_pe = ms.cross_val_predict(params_search.best_estimator_, train_instances_pe, train_labels_pe, n_jobs=3, cv=cv)

print()
print('First step:\n')
print('- Confusion matrix (TN FP FN TP):', mt.confusion_matrix(train_labels_pe, predicted_labels_pe).ravel())
print("- AUC-ROC: %0.2f" % (mt.roc_auc_score(train_labels_pe, predicted_labels_pe)*100))
print("- Accuracy: %0.2f" % (mt.accuracy_score(train_labels_pe, predicted_labels_pe)*100))

scores = ms.cross_validate(params_search.best_estimator_, train_instances_pe, train_labels_pe, scoring=scoring, n_jobs=3, cv=cv)

print()
print('Second step:\n')
print("- AUC-ROC:  %0.2f (+/- %0.2f)" % (scores['test_roc_auc'].mean()*100, scores['test_roc_auc'].std()*100))
print("- Accuracy: %0.2f (+/- %0.2f)" % (scores['test_accuracy'].mean()*100, scores['test_accuracy'].std()*100))


#---


print('\nSP ---')

params_search = RandomizedSearchCV(estimator=clf, param_distributions=param_grid, n_iter=cv, scoring='roc_auc', n_jobs=3, cv=(cv*cv))
params_search.fit(train_instances_sp, train_labels_sp)

print()
print('Best parameters:\n')
print(params_search.best_params_)

predicted_labels_sp = ms.cross_val_predict(params_search.best_estimator_, train_instances_sp, train_labels_sp, cv=cv, n_jobs=-1)

print()
print('First step:\n')
print('- Confusion matrix (TN FP FN TP):', mt.confusion_matrix(train_labels_sp, predicted_labels_sp).ravel())
print("- AUC-ROC: %0.2f" % (mt.roc_auc_score(train_labels_sp, predicted_labels_sp)*100))
print("- Accuracy: %0.2f" % (mt.accuracy_score(train_labels_sp, predicted_labels_sp)*100))

scores = ms.cross_validate(params_search.best_estimator_, train_instances_sp, train_labels_sp, cv=cv, n_jobs=-1, scoring=scoring)

print()
print('Second step:\n')
print("- AUC-ROC:  %0.2f (+/- %0.2f)" % (scores['test_roc_auc'].mean()*100, scores['test_roc_auc'].std()*100))
print("- Accuracy: %0.2f (+/- %0.2f)" % (scores['test_accuracy'].mean()*100, scores['test_accuracy'].std()*100))



Parameter names currently in use by RF:

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}

PE ---

Best parameters:

{'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'criterion': 'entropy'}

First step:

- Confusion matrix (TN FP FN TP): [2665  753  276 3142]
- AUC-ROC: 84.95
- Accuracy: 84.95

Second step:

- AUC-ROC:  89.36 (+/- 1.23)
- Accuracy: 84.74 (+/- 1.43)

SP ---

Best parameters:

{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': None, 'criterion': 'entropy'}

First step:

- Confusion matrix (TN FP FN TP): [1543  270  104 1709]
- AUC-ROC: 89.69
- Accuracy: 89